In [20]:
import cv2
import os
import torch
import json
import numpy as np
from tqdm.notebook import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from mmcv.transforms import Compose
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            transformed_images = self.transform(image)
        else:
            transformed_images = image

        return image_id, image_path, transformed_images


# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

In [21]:
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 1000
BATCH_SIZE = 1

ROOT_DATASET_DIR = '/teamspace/studios/this_studio/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH, device=DEVICE)
evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)
eval_data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)

DEVICE

Loads checkpoint by local backend from path: /teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


device(type='cuda', index=0)

In [22]:
from tqdm.notebook import tqdm
import torch

from mmdet.models.utils import samplelist_boxtype2tensor
from mmengine.registry import MODELS
from mmcv.transforms import Compose

test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator.dataset_meta = model.model.dataset_meta
test_evaluator.ann_file = ANNOTATIONS_JSON_PATH
test_evaluator = Compose(test_evaluator)

collate_preprocessor = model.preprocess
predict_by_feat = model.model.bbox_head.predict_by_feat
rescale = True

preprocessor = MODELS.build(model.cfg.model.data_preprocessor)
def add_pred_to_datasample(data_samples, results_list):
    for data_sample, pred_instances in zip(data_samples, results_list):
        data_sample.pred_instances = pred_instances
    samplelist_boxtype2tensor(data_samples)
    return data_samples

loading annotations into memory...
Done (t=0.32s)
creating index...
index created!


In [23]:
def pass_calibration_data(model: torch.nn.Module, samples: int):
    data_loader = eval_data_loader
    batch_size = data_loader.batch_size
    model.eval()
    batch_ctr = 0
    with torch.no_grad():
        for image_id, image_path, _ in tqdm(data_loader):
            pre_processed = collate_preprocessor(inputs=image_path, batch_size=batch_size)
            _, data = list(pre_processed)[0]
            data = preprocessor(data, False)
            
            preds = model(data['inputs'].to(DEVICE))

            batch_ctr += 1
            if (batch_ctr * batch_size) > samples:
                break  

In [24]:
BASE_PATH = "/teamspace/studios/this_studio/aimet/exported_models_0.01_2/quant_scheme_W@tf-enhanced _ A@tf_encodings"

In [25]:
import onnxruntime as ort
import numpy as np

def onnx_infer(session, data):    
    inp_names = [x.name for x in session.get_inputs()]
    out_names = [x.name for x in session.get_outputs()]
    if type(data) == np.ndarray:
        data = [data]
    assert len(inp_names) == len(data)

    input_dict = dict(zip(inp_names, data))
    result = session.run(out_names, input_dict)

    return result

In [26]:
import os.path as osp
model_path = osp.join(BASE_PATH.replace("encodings", "embedded"), "rtm_det_embedded.onnx")

session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
results = onnx_infer(session, dummy_input.detach().cpu().numpy())

for r in results:
    print(r.shape)

(1, 80, 80, 80)
(1, 80, 40, 40)
(1, 80, 20, 20)
(1, 4, 80, 80)
(1, 4, 40, 40)
(1, 4, 20, 20)


In [27]:
from glob import glob
from mmengine.structures import InstanceData
import onnxruntime as ort

def eval_callback(model_path, save_dir="./temp"):
    data_loader = eval_data_loader
    new_preds = []
    session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    for image_id, image_path, _ in tqdm(data_loader):
        pre_processed = collate_preprocessor(inputs=image_path, batch_size=BATCH_SIZE)
        _, data = list(pre_processed)[0]
        data = preprocessor(data, False)

        preds = onnx_infer(session, data['inputs'].detach().cpu().numpy())
        preds = tuple([tuple([torch.from_numpy(p) for p in preds[:3]]), tuple([torch.from_numpy(p) for p in preds[3:]])])

        batch_img_metas = [
        data_samples.metainfo for data_samples in data['data_samples']
        ]
        preds = predict_by_feat(*preds, batch_img_metas=batch_img_metas, rescale=True)
        preds = add_pred_to_datasample(data['data_samples'], preds)
        
        for img_id, pred in zip(image_id, preds):
            pred = pred.pred_instances
            new_pred = InstanceData(metainfo={"img_id": int(img_id)})
            new_pred.bboxes = [np.array(p) for p in pred['bboxes'].cpu()]
            new_pred.labels = pred['labels'].cpu()
            new_pred.scores = pred['scores'].cpu()
            new_preds.append(new_pred)

    eval_results = test_evaluator(new_preds)
    
    os.makedirs(save_dir, exist_ok=True)
    num_file = len(glob(f"{save_dir}/onnx_acc_*"))
    print("Saving in this path: ", f"{save_dir}/onnx_acc_{num_file}.json")
    with open(f"{save_dir}/onnx_acc_{num_file}.json", "w") as f:
        json.dump(eval_results, f, indent=4)
    bbox_map = eval_results['bbox_mAP']
    return bbox_map


In [28]:
save_dir = BASE_PATH.replace("encodings", "embedded")
model_path = osp.join(BASE_PATH.replace("encodings", "embedded"), "rtm_det_embedded.onnx")

save_dir, model_path

('/teamspace/studios/this_studio/aimet/exported_models_0.01_2/quant_scheme_W@tf-enhanced _ A@tf_embedded',
 '/teamspace/studios/this_studio/aimet/exported_models_0.01_2/quant_scheme_W@tf-enhanced _ A@tf_embedded/rtm_det_embedded.onnx')

In [29]:
eval_callback(model_path, save_dir)

  0%|          | 0/5000 [00:00<?, ?it/s]

08/23 10:54:34 - mmengine - INFO - Evaluating bbox...
Loading and preparing results...
DONE (t=3.74s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=76.32s).
Accumulating evaluation results...
DONE (t=22.68s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.290
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.431
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.311
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.138
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.275
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.467
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.519
 Average Rec

0.29